# ID2214/FID3214 Assignment 2 Group no. [ID2214 - 3]
### Project members: 
[Francesco Gelain, gelain@kth.se]
[Borja Javierre, javierre@kth.se]
[Jingyi Hu, jingyihu@kth.se]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [3]:
import numpy as np
import pandas as pd
import time

In [4]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.10.7
NumPy version: 1.23.4
Pandas version: 1.5.1


## Reused functions from Assignment 1

In [5]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def column_filter(df): 

    filtered_df = df.copy() #copy input dataframe

    #iterate through all columns and consider to drop a column only if it is not labeled "CLASS" or "ID"
    #you may check the number of unique (non-missing) values in a column by applying the pandas functions
    #dropna and unique to drop missing values and get the unique (remaining) values
    filtered_df = filtered_df.dropna(how = 'all', axis = 1)
    for col in filtered_df.columns:
        if col != "CLASS" and col != "ID":
            if filtered_df[col].dropna().unique().size == 1:
                filtered_df = filtered_df.drop(col, axis=1)

    column_filter = filtered_df.columns #list of the names of the remaining columns, including "CLASS" and "ID"
    
    return filtered_df, column_filter

def apply_column_filter(df, column_filter):

    filtered_new_df = df.copy() #copy input dataframe

    #drop each column that is not included in column_filter
    for col in filtered_new_df.columns:
        if col not in column_filter:
            filtered_new_df = filtered_new_df.drop(col, axis=1)

    return filtered_new_df

def imputation(df):
    df_temp = df.copy()
    values = {}
    for column in df_temp:
        #print('Column Name : ', column)
        columnSeriesObj = df_temp[column]
        if columnSeriesObj.dtype == int or columnSeriesObj.dtype == float:
             values[column] = columnSeriesObj.mean()
        elif columnSeriesObj.dtype == object:
             values[column] = columnSeriesObj.mode()[0]

    #print(values)
    df_temp.fillna(value=values, inplace=True)

    return df_temp, values

def apply_imputation(df,imputation):
    df_temp = df.copy()
    values = imputation
    #print(values)
    df_temp.fillna(value=values, inplace=True)
    return df_temp

def normalization(df, normalizationtype): # minmax (default) or zscore 

    new_df = df.copy() #copy input dataframe
    normalization = {}

    #a mapping (dictionary) from each column name to a triple, consisting of ("minmax",min_value,max_value) or ("zscore",mean,std)
    #consider columns of type "float" or "int" only (and which are not labeled "CLASS" or "ID")
    if normalizationtype == "minmax":
        for col in new_df.columns:
            if (new_df[col].dtype == "float" or new_df[col].dtype == "int") and col != "CLASS" and col !="ID":
                #normalization[col] = ({"CLASS": ("minmax", 0, 1), "ID": ("minmax", 0, 1)})
                normalization[col] = ("minmax", new_df[col].min(), new_df[col].max())
                new_df[col] = [(x-new_df[col].min())/(new_df[col].max()-new_df[col].min()) for x in new_df[col]]
    elif normalizationtype == "zscore":
        for col in new_df.columns:
            if (new_df[col].dtype == "float" or new_df[col].dtype == "int") and col != "CLASS" and col !="ID":
                #normalization[col] = ({"CLASS": ("zscore", 0, 1), "ID": ("zscore", 0, 1)})
                normalization[col] = ("zscore", new_df[col].mean(), new_df[col].std())
                new_df[col] = new_df[col].apply(lambda x:(x-new_df[col].mean())/new_df[col].std())

    return new_df, normalization

def apply_normalization(df, normalization):

    new_df = df.copy() #copy input dataframe
    
    for col in new_df.columns:
        if (new_df[col].dtype == "float" or new_df[col].dtype == "int") and col != "CLASS" and col !="ID":
            if normalization[col][0] == "minmax":
                new_df[col] = (new_df[col] - normalization[col][1])/(normalization[col][2] - normalization[col][1])
            elif normalization[col][0] == "zscore":
                new_df[col] = (new_df[col] - normalization[col][1])/normalization[col][2]

    return new_df

def one_hot(df):

    new_df = df.copy() #copy input dataframe

    one_hot = {} #a mapping (dictionary) from column name to a set of categories (possible values for the feature)

# if (new_df[col].dtype == "object" or new_df[col].dtype == "category") and col != "CLASS" and col !="ID":
    for col in new_df.columns:
        if new_df[col].dtype == "category" and col != "CLASS" and col !="ID":
            one_hot[col] = set(new_df[col])
            for value in one_hot[col]:
                new_df[col + "_" + value] = (new_df[col] == value).astype(float)
            new_df = new_df.drop(col, axis=1)

    return new_df, one_hot

def apply_one_hot(df, one_hot):
    
    new_df = df.copy() #copy input dataframe

    for col in new_df.columns:
        if new_df[col].dtype == "category" and col != "CLASS" and col !="ID":
            for value in one_hot[col]:
                new_df[col + "_" + value] = (new_df[col] == value).astype(float)
            new_df = new_df.drop(col, axis=1)

    return new_df

def accuracy(df, correctlabels):
    df_temp = df.copy()
    count = 0
    outputlabels = df_temp.idxmax(axis = 1)
    for i in range(outputlabels.size):
        if correctlabels[i] == outputlabels[i]:
            count += 1
    accuracy = count/outputlabels.size
        
    return accuracy

def brier_score(df, correctlabels):
    df_temp = df.copy()
    brier_score = 0
    mean = 0
    df_correct = pd.DataFrame(np.zeros((len(df), len(np.unique(correctlabels)))), columns=np.unique(correctlabels))
    for i in range(len(correctlabels)):
        df_correct.loc[i, correctlabels[i]] = 1
    #print(df_correct)
    for column in df_correct:
        columnSeriesObj = df_correct[column]
        for i in range(columnSeriesObj.size):
            brier_score += (df_correct.loc[i, column] - df_temp.loc[i, column])**2
    brier_score = brier_score/len(df)
    return brier_score

def auc(df, correctlabels):
    df_temp = df.copy()
    values = {}
    auc_percolumn = 0
    df_truepositive = pd.DataFrame(np.zeros((len(df_temp), len(df_temp.columns))), columns=df_temp.columns)
    for i in range(len(correctlabels)):
        df_truepositive.loc[i, correctlabels[i]] = 1
    #print("df_truepositive: ", df_truepositive)

    for column in df_temp:
        columnseriesobj = df_temp[column]
        columntruepositive = df_truepositive[column]
        columnfalsepositive = columntruepositive.copy()
        for i in range(len(columntruepositive)):
            if columntruepositive[i] == 0:
                columnfalsepositive[i] = 1
            elif columntruepositive[i] == 1:
                columnfalsepositive[i] = 0
        df_auc_temp = pd.DataFrame({"s": columnseriesobj, "tp": columntruepositive, "fp": columnfalsepositive})        
        
        agg_functions = {'tp': 'sum', 'fp': 'sum'}
        df_auc_temp = df_auc_temp.groupby(df_auc_temp['s']).aggregate(agg_functions)
        df_auc_temp = df_auc_temp.sort_values(by = 's', ascending = False)

        # return auc_score
        AUC = 0
        cov_tp = 0
        tot_tp = np.sum(columntruepositive)
        tot_fp = np.sum(columnfalsepositive)

        for idx in df_auc_temp.index:
            if df_auc_temp["fp"][idx] == 0:
                cov_tp += df_auc_temp["tp"][idx]
            elif df_auc_temp["tp"][idx] == 0:
                AUC += (cov_tp / tot_tp) * (df_auc_temp["fp"][idx] / tot_fp)
            else:
                AUC += (cov_tp / tot_tp) * (df_auc_temp["fp"][idx] / tot_fp) + (df_auc_temp["tp"][idx] / tot_tp) * (
                    df_auc_temp["fp"][idx] / tot_fp) / 2
                cov_tp += df_auc_temp["tp"][idx]

        #AUC = metrics.roc_auc_score(columntruepositive, columnseriesobj)
        values[column] = AUC

    #print(values)

    auc_score = 0
    for i in values:
        count = 0
        for output in correctlabels:
            if i == output:
                count += 1
        frequency = count / len(correctlabels)
        auc_score += values[i] * frequency
    return auc_score


def euclidean(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist


def create_bins(df,nobins = 10,bintype = 'equal-width'): # defining the function
    df_temp = df.copy() # creating a copy of the input dataFrame
    binning = {} 
    for column in df_temp: # running trough the colums in the copy of the input dataFrame
        # selecting only the requested columns
        if df_temp[column].dtype == 'int64' or 'float64' or 'int32' or 'float32':
            if column != "CLASS" and column != "ID":
                # differentiating for the two cases equal-width and equal-size, using pd.cut and pd.qcut functions respectively 
                if bintype == 'equal-width':
                    cat, bins = pd.cut(df_temp[column],bins=nobins,retbins=True,labels=False)
                elif bintype == 'equal-size':
                    cat, bins = pd.qcut(df_temp[column],q=nobins,retbins=True,labels=False,duplicates='drop')
                # replacing the extremes of the bins with the required -np.inf and np.inf
                bins[0] = -np.inf
                bins[-1] = np.inf
                # filling the dictionary and the dataFrame
                binning[column] = bins
                df_temp[column] = cat.astype('category')
    return df_temp,binning

def apply_bins(df,binning):
    df_temp = df.copy()
    for column in df_temp:
        if df_temp[column].dtype == 'int64' or 'float64' or 'int32' or 'float32':
            if column != "CLASS" and column != "ID":
                cat, bins = pd.cut(df_temp[column],bins=binning[column],retbins=True,labels=False)
                df_temp[column] = cat.astype('category')
    return df_temp

def getfrequencies(classlabels):
    totalcount = classlabels.value_counts().to_dict()
    totalindex = sum(totalcount.values())
    #total = sum(my_dict.values())
    result = {key: value / totalindex for key, value in totalcount.items()}
    return result


## 1. Define the class kNN

In [6]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

# kNN class definition
class kNN:
    
    # __init__ function definition
    def __init__(self):
        # initializing the following attributes of the object (self) to None
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
        self.training_time = None

    # fit function definition
    def fit(self, df, normalizationtype="minmax"):
        # applying column filtering, imputation, normalization and one-hot
        # assigning values to the attributes of the object (self)
        df_filter, column_fil = column_filter(df)
        self.column_filter = column_fil
        df_imputation, imp = imputation(df_filter)
        self.imputation = imp
        df_norm, norm = normalization(df_imputation, normalizationtype)
        self.normalization = norm
        df_onehot, oh = one_hot(df_norm)
        self.one_hot = oh
        self.training_labels = df["CLASS"].astype("category")
        self.labels = self.training_labels.cat.categories
        df_onehot = df_onehot.drop(["CLASS", "ID"], axis=1) # dropping CLASS and ID columns
        self.training_data = df_onehot.values
        self.classes = sorted(df["CLASS"].unique())
    
    # get_nearest_neighbor_predictions function definition
    def get_nearest_neighbor_predictions(self, x_test, k):
        labels = []
        probabilities = np.zeros((len(x_test), len(self.classes)))

        for item in x_test: 
            euclidean_dist = []
            # calculating the euclidean distance the items of the test set and the rows of the training set
            for j in range(len(self.training_data)): 
                euclidean_dist.append(euclidean(item, np.array(self.training_data[j,:]))) 
            euclidean_dist = np.array(euclidean_dist) 
            # sorting the array and keeping only the first K elements
            dist = np.argsort(euclidean_dist)[:k] 
            temp_labels = []
            # storing the class for the selected elements
            for i in dist:
                temp_labels.append(self.training_labels[i])
            labels.append(temp_labels)
  
        for i in range(len(x_test)):
            for n in range(k):
                for m in range(len(self.classes)):
                    if labels[i][n] == self.classes[m]:
                        probabilities[i][m] += 1            
        df_probs = pd.DataFrame(probabilities, columns= self.classes)
        
        for col, val in df_probs.iteritems():
            df_probs[col] = df_probs[col].apply(lambda x: x/k)
        
        return df_probs
    
    # predict function definition
    def predict(self, df, k):
        dfcpy = df.copy()

        #filters from fit function
        df_filtered = apply_column_filter(dfcpy,self.column_filter)
        df_imputed = apply_imputation(df_filtered, self.imputation)
        df_normalized = apply_normalization(df_imputed, self.normalization)
        df_onehot = apply_one_hot(df_normalized, self.one_hot)
        df_onehot = df_onehot.drop(['CLASS','ID'], axis=1)
        self.test_data = df_onehot.values

        predictions = self.get_nearest_neighbor_predictions(self.test_data, k)
        # display(predictions) check
        return predictions

In [7]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.46 s.


/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/107655361.py:113: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, val in df_probs.iteritems():


Testing time (k=1): 0.27 s.


/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/107655361.py:113: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, val in df_probs.iteritems():


Testing time (k=3): 0.58 s.


...

/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/107655361.py:113: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, val in df_probs.iteritems():


Testing time (k=7): 0.29 s.
Testing time (k=9): 0.28 s.



/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/107655361.py:113: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, val in df_probs.iteritems():


'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.474019,0.833805
7,0.598131,0.470723,0.834465
9,0.616822,0.483674,0.828734


In [8]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/107655361.py:113: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, val in df_probs.iteritems():


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [14]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from 
#         a column name (a specific feature) to another mapping, which given a class label and a value for 
#         the feature, returns the number of training instances which have included this combination, 
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you 
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.

def gettraininginstances1(self, training_features): #this is a mapping of a mapping, i.e. a dictionary in a dictionary

    values = {} #this dictionary maps the colname with the values in the columns 
    for column_name in training_features: 
        values[column_name] = training_features[column_name].unique() #get all unique values in the column

    result_mapping = {} #this dicctionary maps the column name to combination of value and class label
    for column_name in training_features: #go throughe every column and its values
        if column_name != "CLASS" or column_name != "ID": #if the column is not the class label or the ID column 
            continue
        else:
            mapping = {} #mapping of value and class label to number of training instances 
            allin = training_features.groupby([column_name, "CLASS"]) #get all combinations of class label and value in the column
            count_training_features = allin.size() #get the size of the combination
        
        for vals in values[column_name]: #go through the values in every column
            for label in self.labels: #go though all the labels
                mapping[(vals, label)] = len(allin.get_group(vals, label)) #get the number of training instances with the combination of value and class label
        result_mapping[column_name] = mapping #add the mapping to the result mapping
    
    return result_mapping

def gettraininginstances2(self, training_features):
    
    values = {} #this dictionary maps the colname with the values in the columns 
    for column_name in training_features: 
        values[column_name] = training_features[column_name].unique() #get all unique values in the column

    result_mapping = {} #this dicctionary maps the column name to combination of value and class label
    for column_name in training_features: #go throughe every column and its values
        if column_name != "CLASS" or column_name != "ID": #if the column is not the class label or the ID column 
            continue
        else:
            mapping = {} #mapping of value and class label to number of training instances 
            allin = training_features.groupby([column_name, "CLASS"]) #get all combinations of class label and value in the column
            count_training_features = allin.size() #get the size of the combination
        
        for vals in values[column_name]: #go through the values in every column
            mapping[vals] = len(allin.get_group(vals)) #get the number of training instances for value 
        result_mapping[column_name] = mapping #add the mapping to the result mapping
    
    return result_mapping

class NaiveBayes:
    def _init_(self):
        self.column_filter = None
        self.binning = None
        self.labels = None
        self.class_priors = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None
    
    def fit(self, df, nobins=10, bintype="equal-width"): # bintype - either "equal-width" (default) or "equal-size"
        
        #print(df) #print the DataFrame to see its structure
        df_col, self.column_filter = column_filter(df)
        df_bin, self.binning = create_bins(df_col, nobins, bintype)
        #print(df_bin)

        training_labels = df["CLASS"].astype("category")
        self.labels = training_labels.cat.categories
        self.class_priors = getfrequencies(training_labels)

        training_features1 = df.copy() #this will be used for the features (column names)
        self.feature_class_value_counts = gettraininginstances1(self, training_features1)

        training_features2 = df.copy() #this will be used for the features (column names) 
        self.feature_class_counts = gettraininginstances2(self, training_features2) 
        #df.groupby(["CLASS"]).size().to_dict()

    def predict(self, df):
        df = apply_column_filter(df,self.column_filter)
        df = apply_bins(df,self.binning)
        predictions = pd.DataFrame(columns=self.labels)
        for index, row in df.iterrows():
            class_probabilities = {}
            for label in self.labels:
                class_probabilities[label] = self.class_priors[label]
                for feature in row.index:
                    if (label,row[feature]) in self.feature_class_value_counts:
                        class_probabilities[label] *= self.feature_class_value_counts[(label,row[feature])] / self.feature_class_counts[label]
                    else:
                        class_probabilities[label] = 0
            sum_probabilities = sum(class_probabilities.values())
            if sum_probabilities == 0:
                for label in self.labels:
                    class_probabilities[label] = self.class_priors[label]
            else:
                for label in self.labels:
                    class_probabilities[label] /= sum_probabilities
            predictions = predictions.append(class_probabilities,ignore_index=True)
        return predictions

In [15]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.04 s.


/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is d

Testing time (3, 'equal-width'): 0.51 s.
Training time (3, 'equal-size'): 0.12 s.


/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is d

Testing time (3, 'equal-size'): 0.57 s.
Training time (5, 'equal-width'): 0.08 s.


...

Testing time (10, 'equal-width'): 0.28 s.
Training time (10, 'equal-size'): 0.04 s.


/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/3724861004.py:156: FutureWarning: The frame.append method is d

Testing time (10, 'equal-size'): 0.29 s.



'results'

Accuracy  Brier score  AUC
3  equal-width  0.392523     0.719364  0.5
   equal-size   0.392523     0.719364  0.5
5  equal-width  0.392523     0.719364  0.5
   equal-size   0.392523     0.719364  0.5
10 equal-width  0.392523     0.719364  0.5
   equal-size   0.392523     0.719364  0.5

In [13]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/4072627887.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/4072627887.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/4072627887.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/4072627887.py:142: FutureWarning: The frame.append method is d

Accuracy on training set: 0.3178
AUC on training set: 0.5000
Brier score on training set: 0.7658


/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/4072627887.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/4072627887.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/4072627887.py:142: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predictions = predictions.append(class_probabilities,ignore_index=True)
/var/folders/vd/s9rj2zys3hs8hf8cb8lr9lfr0000gn/T/ipykernel_67003/4072627887.py:142: FutureWarning: The frame.append method is d

### Comment on assumptions, things that do not work properly, etc.